In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import random
import numpy as np
import torch
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from torch.utils.data import Dataset, DataLoader

In [ ]:


# ------------------------------------------------------------------------
# 1) Read & Sample the Dataset (5% in a balanced way)
# ------------------------------------------------------------------------
df = pd.read_parquet("hf://datasets/TimKoornstra/financial-tweets-sentiment/data/train-00000-of-00001.parquet")

label_col = "sentiment"  # or your actual label column
text_col  = "tweet"      # or your actual text column

# Balanced sampling (5% per class)
df_small = df.groupby(label_col, group_keys=False).apply(
    lambda x: x.sample(frac=0.05, random_state=42)
).reset_index(drop=True)

print("Original dataset size:", len(df))
print("Reduced (5%) dataset size:", len(df_small))
print("Sampled class distribution:")
print(df_small[label_col].value_counts())

# ------------------------------------------------------------------------
# 2) Define Hyperparameter Grid
# ------------------------------------------------------------------------
models = [
    "ProsusAI/finbert",         # FinBERT
    "prajjwal1/bert-tiny",      # BERT Tiny
    "vinai/bertweet-base",      # BERTweet
    "cardiffnlp/twitter-roberta-base-sentiment-latest"  # Twitter RoBERTa
]
optimizers = ["adam", "sgd"]  # We'll show how to map these to torch.optim
learning_rates = [1e-5, 3e-5, 5e-5]
epochs_list = [1, 2, 3]
seeds = [42, 1984, 1337]

param_combinations = list(itertools.product(models, optimizers, learning_rates, epochs_list, seeds))

# ------------------------------------------------------------------------
# 3) Create a Custom Dataset & Data Collation
# ------------------------------------------------------------------------
class TextClassificationDataset(Dataset):
    def __init__(self, df, tokenizer, text_col, label_col, max_len=128):
        self.tokenizer = tokenizer
        self.texts = df[text_col].tolist()
        self.labels_raw = df[label_col].tolist()
        self.max_len = max_len

        # You could map labels from strings to IDs if needed:
        # e.g., self.label2id = {label: i for i, label in enumerate(sorted(df[label_col].unique()))}
        # self.labels = [self.label2id[x] for x in self.labels_raw]
        # But if your labels are already integers [0,1,2,...], you can keep them directly:
        # We'll just auto-detect if they look like strings or ints:

        unique_labels = sorted(list(set(self.labels_raw)))
        self.label2id = {lab: i for i, lab in enumerate(unique_labels)}
        self.labels = [self.label2id[l] for l in self.labels_raw]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"  # returns dict of tensors
        )
        # encoding["input_ids"] has shape [1, max_len], so we will squeeze(0).
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item["labels"] = torch.tensor(label, dtype=torch.long)

        return item

def create_dataloader(df_part, tokenizer, text_col, label_col, batch_size=16, shuffle=True):
    ds = TextClassificationDataset(df_part, tokenizer, text_col, label_col)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=shuffle)
    return loader, ds.label2id

# ------------------------------------------------------------------------
# 4) Training & Evaluation (Manual Loop)
# ------------------------------------------------------------------------
def train_and_evaluate(model_name, optimizer_name, learning_rate, num_epochs, seed_value, df_data):
    # Set seeds
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 4a) Split data into train/val
    train_df, val_df = train_test_split(
        df_data,
        test_size=0.2,
        random_state=seed_value,
        stratify=df_data[label_col]
    )

    # 4b) Load tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(df_data[label_col].unique())
    )
    model.to(device)

    # 4c) Create data loaders
    train_loader, label2id = create_dataloader(train_df, tokenizer, text_col, label_col, batch_size=16, shuffle=True)
    val_loader, _          = create_dataloader(val_df,   tokenizer, text_col, label_col, batch_size=16, shuffle=False)

    # 4d) Create optimizer
    if optimizer_name.lower() == "adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name.lower() == "sgd":
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # 4e) Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for batch in train_loader:
            # Move data to GPU if available
            input_ids      = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels         = batch["labels"].to(device)

            # Zero out gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            total_loss += loss.item()

            # Backward pass
            loss.backward()
            optimizer.step()

        avg_train_loss = total_loss / len(train_loader)

        # Evaluate on validation set after each epoch
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids      = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels         = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=-1)

                all_preds.extend(preds.cpu().tolist())
                all_labels.extend(labels.cpu().tolist())

        # Compute accuracy
        accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
        print(f"[{model_name}] Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {avg_train_loss:.4f} | Val Accuracy: {accuracy:.4f}")

    # Final validation accuracy after all epochs
    return accuracy

# ------------------------------------------------------------------------
# 5) Run the Grid Search & Collect Results
# ------------------------------------------------------------------------
results = []
for (model_name, opt_name, lr, ep, seed_val) in param_combinations:
    print(f"=== Now training: model={model_name}, optimizer={opt_name}, lr={lr}, epochs={ep}, seed={seed_val} ===")
    accuracy = train_and_evaluate(
        model_name=model_name,
        optimizer_name=opt_name,
        learning_rate=lr,
        num_epochs=ep,
        seed_value=seed_val,
        df_data=df_small
    )

    results.append({
        "model": model_name,
        "optimizer": opt_name,
        "learning_rate": lr,
        "epochs": ep,
        "seed": seed_val,
        "accuracy": accuracy
    })

# ------------------------------------------------------------------------
# 6) Identify the Best Combination
# ------------------------------------------------------------------------
best_result = max(results, key=lambda x: x["accuracy"])

print("\nAll results:")
for r in results:
    print(r)

print("\nBest hyperparameters based on accuracy:")
print(best_result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Original dataset size: 38091
Reduced (5%) dataset size: 1904
Sampled class distribution:
sentiment
1    868
0    609
2    427
Name: count, dtype: int64
=== Now training: model=ProsusAI/finbert, optimizer=adam, lr=1e-05, epochs=1, seed=42 ===


<ipython-input-2-50205a68fcba>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby(label_col, group_keys=False).apply(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

...

[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 1.5058 | Val Accuracy: 0.3675
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 1.2058 | Val Accuracy: 0.3753
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 1.1258 | Val Accuracy: 0.3990
=== Now training: model=cardiffnlp/twitter-roberta-base-sentiment-latest, optimizer=sgd, lr=5e-05, epochs=3, seed=1984 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 1.5202 | Val Accuracy: 0.3937
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 1.2082 | Val Accuracy: 0.3858
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 1.1262 | Val Accuracy: 0.3937
=== Now training: model=cardiffnlp/twitter-roberta-base-sentiment-latest, optimizer=sgd, lr=5e-05, epochs=3, seed=1337 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 1.5323 | Val Accuracy: 0.3806
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 1.2026 | Val Accuracy: 0.3780
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 1.1209 | Val Accuracy: 0.4042

All results:
{'model': 'ProsusAI/finbert', 'optimizer': 'adam', 'learning_rate': 1e-05, 'epochs': 1, 'seed': 42, 'accuracy': 0.5249343832020997}
{'model': 'ProsusAI/finbert', 'optimizer': 'adam', 'learning_rate': 1e-05, 'epochs': 1, 'seed': 1984, 'accuracy': 0.5328083989501312}
{'model': 'ProsusAI/finbert', 'optimizer': 'adam', 'learning_rate': 1e-05, 'epochs': 1, 'seed': 1337, 'accuracy': 0.4776902887139108}
{'model': 'ProsusAI/finbert', 'optimizer': 'adam', 'learning_rate': 1e-05, 'epochs': 2, 'seed': 42, 'accuracy': 0.5538057742782152}
{'model': 'ProsusAI/finbert', 'optimizer': 'adam', 'learning_rate': 1e-05, 'epochs': 2, 'seed': 1984, 'accuracy': 0.5590551181102362}
{'mod

In [ ]:
def find_best_seed_for_fixed_params(
    model_name,
    optimizer_name,
    learning_rate,
    num_epochs,
    df_data,
    n_seeds=100,
    seed_range=(0, 9999)
):
    """
    This function picks random seeds within `seed_range` (default 0..9999),
    samples n_seeds of them, and tries each seed with the same model/optimizer/hparams.
    Returns the best seed + accuracy found.
    """
    import random

    # Generate "n_seeds" unique random seeds from the given range.
    # If you prefer consecutive seeds 0..99, you can just do range(n_seeds).
    candidate_seeds = random.sample(range(seed_range[0], seed_range[1]+1), n_seeds)

    seed_scores = []
    best_score = float('-inf')
    best_seed = None

    for s in candidate_seeds:
        print(f"\n=== Training with seed={s} ===")
        accuracy = train_and_evaluate(
            model_name=model_name,
            optimizer_name=optimizer_name,
            learning_rate=learning_rate,
            num_epochs=num_epochs,
            seed_value=s,
            df_data=df_data
        )
        seed_scores.append({"seed": s, "accuracy": accuracy})

        # Track the best so far
        if accuracy > best_score:
            best_score = accuracy
            best_seed = s

    print("\nAll seed results:")
    for item in seed_scores:
        print(item)

    print(f"\nBest seed out of {n_seeds} tested: {best_seed} with accuracy: {best_score:.4f}")
    return best_seed, best_score, seed_scores

# ------------------------------------------------------------------------
# Example usage:
# We already know the best model + optimizer + lr + epochs from your search.
best_model   = "cardiffnlp/twitter-roberta-base-sentiment-latest"
best_opt     = "adam"
best_lr      = 5e-5
best_epochs  = 3

best_seed_over_100, best_acc_over_100, seed_experiments = find_best_seed_for_fixed_params(
    model_name    = best_model,
    optimizer_name= best_opt,
    learning_rate = best_lr,
    num_epochs    = best_epochs,
    df_data       = df_small,       # The 5% sampled dataset
    n_seeds       = 100,            # Number of different seeds to try
    seed_range    = (0, 9999)       # or any other range you prefer
)



=== Training with seed=7871 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 0.9598 | Val Accuracy: 0.6325
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 0.6763 | Val Accuracy: 0.7192
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 0.3621 | Val Accuracy: 0.7375

=== Training with seed=1026 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 0.9537 | Val Accuracy: 0.5984
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 0.6626 | Val Accuracy: 0.6955
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 0.3773 | Val Accuracy: 0.7192

=== Training with seed=4217 ===


...

[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 0.9792 | Val Accuracy: 0.6194
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 0.7119 | Val Accuracy: 0.6798
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 0.3981 | Val Accuracy: 0.6719

=== Training with seed=3259 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 0.9243 | Val Accuracy: 0.6614
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 0.6476 | Val Accuracy: 0.7244
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 0.3373 | Val Accuracy: 0.7638

=== Training with seed=5265 ===


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 1/3 | Train Loss: 0.9239 | Val Accuracy: 0.5223
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 2/3 | Train Loss: 0.6533 | Val Accuracy: 0.7008
[cardiffnlp/twitter-roberta-base-sentiment-latest] Epoch 3/3 | Train Loss: 0.3293 | Val Accuracy: 0.7165

All seed results:
{'seed': 7871, 'accuracy': 0.7375328083989501}
{'seed': 1026, 'accuracy': 0.7191601049868767}
{'seed': 4217, 'accuracy': 0.7611548556430446}
{'seed': 3699, 'accuracy': 0.6929133858267716}
{'seed': 8085, 'accuracy': 0.7165354330708661}
{'seed': 8902, 'accuracy': 0.800524934383202}
{'seed': 9236, 'accuracy': 0.7349081364829396}
{'seed': 61, 'accuracy': 0.7165354330708661}
{'seed': 23, 'accuracy': 0.7454068241469817}
{'seed': 6538, 'accuracy': 0.7322834645669292}
{'seed': 5631, 'accuracy': 0.7165354330708661}
{'seed': 2358, 'accuracy': 0.7611548556430446}
{'seed': 7897, 'accuracy': 0.6876640419947506}
{'seed': 8534, 'accuracy': 0.7664041994750657}
{'seed': 2395